<a href="https://colab.research.google.com/github/yayra/Business-Analytics/blob/main/RFM_Retention_Analysis/RFM_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Import libraries**


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


###  **1. Upload file from Github**


In [3]:
df=pd.read_csv("https://raw.githubusercontent.com/yayra/Business-Analytics/refs/heads/main/RFM_Retention_Analysis/rfm_raw_data.csv")
df.head()

,mem_no,last_ord_dt,recency,frequency,monetary,is_back
0,1,01/01/2024,30,1,287000,0
1,2,12/01/2024,19,7,269290,0
2,3,01/01/2024,30,1,548000,0
3,4,27/01/2024,4,3,257000,1
4,5,01/01/2024,30,2,65000,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   mem_no       1000 non-null   int64 
 1   last_ord_dt  1000 non-null   object
 2   recency      1000 non-null   int64 
 3   frequency    1000 non-null   int64 
 4   monetary     1000 non-null   int64 
 5   is_back      1000 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 47.0+ KB


In [5]:
#Display descriptive statistics of the dataset
df.describe()

,mem_no,recency,frequency,monetary,is_back
count,1000.000000,1000.00000,1000.000000,1.000000e+03,1000.000000
mean,500.500000,20.92500,2.439000,2.744007e+05,0.418000
std,288.819436,11.20897,2.801094,1.424759e+05,0.493477
min,1.000000,0.00000,1.000000,6.500000e+04,0.000000
25%,250.750000,9.00000,1.000000,1.954575e+05,0.000000
50%,500.500000,30.00000,2.000000,2.490000e+05,0.000000
75%,750.250000,30.00000,3.000000,3.218750e+05,1.000000
max,1000.000000,30.00000,34.000000,2.450000e+06,1.000000


### **2. Data Manipulation**


In [6]:
#Copy the dataset to do manipulation
df1 = df.copy()

In [7]:
df1['mem_no'].duplicated().sum()

0

In [8]:
#Check the values where recency is equal to 30 and they are indicated as returning customer (eg. 'is_back" is equal 1)
df1[(df1['recency']==30) & (df1['is_back']==1)]

,mem_no,last_ord_dt,recency,frequency,monetary,is_back
38,39,01/01/2024,30,1,390000,1
39,40,01/01/2024,30,1,520000,1
43,44,01/01/2024,30,1,320000,1
45,46,01/01/2024,30,1,402000,1
60,61,01/01/2024,30,1,270000,1
...,...,...,...,...,...,...
969,970,01/01/2024,30,1,229000,1
971,972,01/01/2024,30,1,390000,1
989,990,01/01/2024,30,1,550000,1
994,995,01/01/2024,30,1,350000,1


In [9]:
#Replace 1 with 0 values of 'is_back' columns where recency is equal to 30
df1.loc[df['recency']==30, 'is_back'] = 0
df1[(df['recency']==30) & (df['is_back']==1)]

,mem_no,last_ord_dt,recency,frequency,monetary,is_back
38,39,01/01/2024,30,1,390000,0
39,40,01/01/2024,30,1,520000,0
43,44,01/01/2024,30,1,320000,0
45,46,01/01/2024,30,1,402000,0
60,61,01/01/2024,30,1,270000,0
...,...,...,...,...,...,...
969,970,01/01/2024,30,1,229000,0
971,972,01/01/2024,30,1,390000,0
989,990,01/01/2024,30,1,550000,0
994,995,01/01/2024,30,1,350000,0


In [10]:
# Check for inconsistent data in the 'recency' and 'is_back' columns
# (if 'recency' is equal to 0, 'is_back' cannot be 0)
mask = df1[(df1['is_back']==0) & (df1['recency']==0)]
mask

,mem_no,last_ord_dt,recency,frequency,monetary,is_back
540,541,31/01/2024,0,4,194750,0
623,624,31/01/2024,0,2,241500,0
688,689,31/01/2024,0,2,254500,0
723,724,31/01/2024,0,2,132500,0


In [11]:
# Set the value of column 'is_back' to 1 to achieve data consistency
df1.iloc[mask.index, 5] = 1

Descriptive statistics indicate that the 'frequency' column contains outliers. To identify these outliers more clearly, it is essential to visualize the data using a box plot. Additionally, outliers should be removed using the Interquartile Range (IQR) method to prepare data for logistic regression.


In [12]:
#Check for the outliers in 'frequency' column with a box plot
fig = px.box(df1, x= "frequency",
              width = 600, height = 400,
              template = 'plotly_white')
fig.show()

In [13]:
 # Compute the 25th percentile value in `frequency`
 percentile25 = df1['frequency'].quantile(0.25)
 # Compute the 75th percentile value in `frequency`
 percentile75 = df1['frequency'].quantile(0.75)
 # Compute the interquartile range in `frequency`
 iqr = percentile75 - percentile25

# Define the upper limit and lower limit for non-outlier values in `frequency`
 upper_limit = percentile75 + 1.5 * iqr
 lower_limit = percentile25 - 1.5 * iqr
 print("Lower limit:", lower_limit)
 print("Upper limit:", upper_limit)

 # Identify subset of data containing outliers in `frequency`
 outliers = df1[(df1['frequency'] > upper_limit) | (df1['frequency'] < lower_limit)]
 # Count how many rows in the data contain outliers in `frequency`
 print("Number of rows in the data containing outliers in `frequency`:", len(outliers))

Lower limit: -2.0
Upper limit: 6.0
Number of rows in the data containing outliers in `frequency`: 50


In [14]:
# Drop outliers
df1 = df1.drop(outliers.index)
df1.describe()

,mem_no,recency,frequency,monetary,is_back
count,950.000000,950.000000,950.000000,9.500000e+02,950.000000
mean,502.082105,21.774737,1.958947,2.760000e+05,0.260000
std,288.819598,10.735727,1.225562,1.439939e+05,0.438865
min,1.000000,0.000000,1.000000,6.500000e+04,0.000000
25%,251.250000,11.000000,1.000000,1.966700e+05,0.000000
50%,503.500000,30.000000,2.000000,2.500000e+05,0.000000
75%,752.750000,30.000000,3.000000,3.242475e+05,1.000000
max,1000.000000,30.000000,6.000000,2.450000e+06,1.000000


### **3. EDA**

In [15]:
recency_cnt = df1.groupby(['is_back', 'recency'])[['mem_no']].count().reset_index()
recency_cnt['is_back'] = recency_cnt['is_back'].astype(str)

fig = px.bar(recency_cnt, x='recency', y='mem_no', color='is_back',
             width=800, height=500,
             template='plotly_white',
             color_discrete_sequence=['#EF553B','#636EFA'],  # Blue and Red
             labels={'mem_no': 'Customer number', 'recency': 'Recency', 'is_back': 'Returning Status'},
             title='Customer Count by Recency')

fig.show()

The graph suggests that most of the returning customers made their purchase within the last 10 days, whereas 511 customers who had a recency of 30 days did not return in the following month. This segment of customers should be analyzed further to understand the factors contributing to their churn.

In [16]:
fig = px.histogram(df1, x='monetary', color='is_back',
                   title='Customers Spending',
                   color_discrete_sequence=['#EF553B','#636EFA'],
                   width = 1000, height = 600,
                   barmode='group',
                   template='plotly_white')
fig.show()

In [17]:
fig = px.box(df1, y='monetary', color='is_back',
             width = 800, height = 500,
             template = 'plotly_white')
fig.show()

In [18]:
df1[df1['is_back'] == 0]['monetary'].describe()


,monetary
count,7.030000e+02
mean,2.793626e+05
std,1.569004e+05
min,6.500000e+04
25%,1.920000e+05
50%,2.500000e+05
75%,3.285000e+05
max,2.450000e+06


In [19]:
df1[df1['is_back'] == 1]['monetary'].describe()

,monetary
count,247.000000
mean,266429.352227
std,98041.531417
min,94500.000000
25%,200000.000000
50%,246670.000000
75%,304500.000000
max,777500.000000


From the provided spending data for returning and churned customers, we can infer several insights:

**1. Spending Behavior Differences:**

**Returning Customers:**

Minimum Spending: 94,500 UZS (higher than churned customers).
Returning customers tend to have a higher baseline spending, indicating they might perceive more value in the product/service and are willing to spend more from the start.

**Churned Customers:**

Minimum Spending: 65,000 UZS (lower than returning customers).
Lower spending suggests churned customers may have had lower initial engagement or satisfaction levels, making them more likely to leave.

**2. Spending Variability:**

**Returning Customers:**
Standard Deviation: 98,041 UZS (lower variability).
More consistent spending patterns suggest that returning customers have stable purchasing behaviors.

**Churned Customers:**

Standard Deviation: 156,900 UZS (higher variability).
High variability indicates that churned customers exhibit inconsistent spending habits. Some may have spent large amounts initially but didn't return, while others may have spent very little and left early. This inconsistency could suggest unmet expectations or fluctuating demand.

### **4. Build a logistic regression model**
 Build a logistic regression model to determine the regression coefficients that measure the influence of each RFM factor on the probability of customer churn.

#### **Correlation**
Before developing a logistic regression model, it is essential to assess the correlation between the independent variables and the dependent variable to identify features that may significantly impact the model's coefficients.

In [20]:
rfm_corr = df1[['recency','frequency', 'monetary', 'is_back']].corr()
fig = px.imshow(rfm_corr, text_auto='.2f',
                color_continuous_scale = 'YlorBr',
                width=500, height=500)
fig.show()

The correlation matrix indicates a high correlation between the independent variables—recency (-0.64) and frequency (0.54)—with the dependent variable, is_back. Additionally, there is a strong negative correlation (-0.71) between the independent variables recency and frequency, which suggests the presence of multicollinearity. Multicollinearity makes it challenging to isolate and accurately assess the individual impact of each independent variable on the dependent variable. Therefore, conducting a Variance Inflation Factor (VIF) analysis is necessary to detect and address multicollinearity.

**Variance Inflation Factor (VIF)**

The Variance Inflation Factor (VIF) is a metric used to quantify the degree of multicollinearity in a regression analysis. It helps identify how much the variance of a regression coefficient is inflated due to correlations among independent variables. A VIF value between 1 and 5 indicates moderate correlation among variables. A VIF less then 5 suggests that all features can be included into the model.  

In [21]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = df1[['recency', 'frequency', 'monetary']]  # Independent variables
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
vif_data

,Feature,VIF
0,recency,2.838839
1,frequency,2.204414
2,monetary,4.042610


In [22]:
df1['is_back'].value_counts(normalize=True).reset_index()

,is_back,proportion
0,0,0.74
1,1,0.26


**Stratification**

As we can see the classes are inbalanced. In *logistic regression*, **stratification** is used to ensure that the distribution of the dependent variable is proportionally represented across training and testing datasets. This is particularly important when dealing with imbalanced datasets, where one class significantly outnumbers the other.

**Standardization**

Also, the **Standard Scaler** will be used to normalize the feature values, ensuring that all independent variables are on the same scale. This is important in logistic regression as it helps improve model convergence, prevents certain features from dominating the learning process due to differing magnitudes, and enhances the overall accuracy and interpretability of the model.

**Hyperparameter tuning**

The Logistic Regression model is trained with L2 regularization (Ridge), which helps prevent overfitting and is effective in handling multicollinearity. The 'lbfgs' solver is used, as it is efficient for small datasets and supports L2 regularization. To ensure convergence, the maximum number of iterations is set to 500.

In [23]:
X = df1[['recency', 'frequency', 'monetary']]
y = df1['is_back']

## Split the data into training and test sets while maintaining the proportion of the target variable (stratification).
# The 'random_state=42' ensures reproducibility of results.
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

## Standardize the feature values to have 0 mean and 1 variance, which helps improve model performance
# by ensuring all features are on the same scale.
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#Hyperparameter tuning
#Regularization
#Build logistic regression model and fit the model to the training data
model = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=500, random_state=42).fit(X_train,y_train)

In [24]:
y_pred = model.predict(X_test)
test_acc = accuracy_score(y_test, y_pred)
print("The Accuracy for Test Set is {}".format(test_acc*100))

The Accuracy for Test Set is 80.25210084033614


In [25]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.86      0.88      0.87       176
           1       0.63      0.60      0.61        62

    accuracy                           0.80       238
   macro avg       0.74      0.74      0.74       238
weighted avg       0.80      0.80      0.80       238



#### **Regression model coefficients**

In [26]:
# Display the coefficients of the regression model
coef = pd.DataFrame({'features': X.columns, 'coefficient': model.coef_[0]})

# Disable scientific notation for floating-point numbers
pd.set_option('display.float_format', '{:.6f}'.format)

coef

,features,coefficient
0,recency,-1.217106
1,frequency,0.662923
2,monetary,0.066683


**Interpretation of logistic regression coefficients:**

* **Recency (-1.217):** A higher negative recency value decreases the probability( log-odds) of the event (in this case retention) occurring, meaning customers who recently interacted are less likely to churn.
* **Frequency (0.663):** A higher frequency increases the probability of customer retention, suggesting that more frequent interactions are associated with a higher likelihood of staying.
* **Monetary (0.067):** A positive but small impact on the likelihood of staying, indicating that spending more slightly increases the odds.

### **5. Standardized Scoring**

Converting RFM values into percentiles normalizes the scores, enabling objective comparison of customers.

In customer segmentation, the np.percentile() function in Python is used to categorize each of the RFM metrics. This function divides data into groups based on specified percentage ranks, allowing segmentation based on actual values.

In [27]:
df1.describe()

,mem_no,recency,frequency,monetary,is_back
count,950.000000,950.000000,950.000000,950.000000,950.000000
mean,502.082105,21.774737,1.958947,275999.957895,0.260000
std,288.819598,10.735727,1.225562,143993.930960,0.438865
min,1.000000,0.000000,1.000000,65000.000000,0.000000
25%,251.250000,11.000000,1.000000,196670.000000,0.000000
50%,503.500000,30.000000,2.000000,250000.000000,0.000000
75%,752.750000,30.000000,3.000000,324247.500000,1.000000
max,1000.000000,30.000000,6.000000,2450000.000000,1.000000


In [28]:
#Dividing the 'recency' values into 5 categories for assigning the score
r1, r2, r3, r4 = np.percentile(df1['recency'], [10, 20, 30, 50])
r1, r2, r3, r4


(3.0, 8.0, 16.0, 30.0)

In [29]:
def percent(x):
  if x <= r1:
    return 5
  elif x > r1 and x <= r2:
    return 4
  elif x >2 and x <= r3:
    return 3
  elif x > r3 and x < r4:
    return 2
  else:
    return 1

In [30]:
df1['recency_score'] = df1['recency'].apply(percent)*-coef.iloc[0,1]
df1.head(10)

,mem_no,last_ord_dt,recency,frequency,monetary,is_back,recency_score
0,1,01/01/2024,30,1,287000,0,1.217106
2,3,01/01/2024,30,1,548000,0,1.217106
3,4,27/01/2024,4,3,257000,1,4.868423
4,5,01/01/2024,30,2,65000,0,1.217106
5,6,15/01/2024,16,2,375000,1,3.651317
6,7,22/01/2024,9,4,925750,0,3.651317
8,9,01/01/2024,30,1,325000,0,1.217106
9,10,01/01/2024,30,1,229000,0,1.217106
10,11,23/01/2024,8,2,249000,1,4.868423
11,12,25/01/2024,6,2,185000,1,4.868423


In [31]:
#Dividing the 'frequency' values into 5 categories for assigning the score
f1, f2, f3, f4 = np.percentile(df1['frequency'], [20, 50, 80, 90])
f1, f2, f3, f4

(1.0, 2.0, 3.0, 4.0)

In [32]:
def percent(x):
  if x <= f1:
    return 1
  elif x > f1 and x <= f2:
    return 2
  elif x >f2 and x <= f3:
    return 3
  elif x > f3 and x <= f4:
    return 4
  else:
    return 5

In [33]:
df1['frequency_score'] = df1['frequency'].apply(percent)*coef.iloc[1,1]
df1.head(10)

,mem_no,last_ord_dt,recency,frequency,monetary,is_back,recency_score,frequency_score
0,1,01/01/2024,30,1,287000,0,1.217106,0.662923
2,3,01/01/2024,30,1,548000,0,1.217106,0.662923
3,4,27/01/2024,4,3,257000,1,4.868423,1.988768
4,5,01/01/2024,30,2,65000,0,1.217106,1.325845
5,6,15/01/2024,16,2,375000,1,3.651317,1.325845
6,7,22/01/2024,9,4,925750,0,3.651317,2.651691
8,9,01/01/2024,30,1,325000,0,1.217106,0.662923
9,10,01/01/2024,30,1,229000,0,1.217106,0.662923
10,11,23/01/2024,8,2,249000,1,4.868423,1.325845
11,12,25/01/2024,6,2,185000,1,4.868423,1.325845


In [34]:
#Dividing the 'monetary' values into 5 categories for assigning the score
m1, m2, m3, m4 = np.percentile(df1['monetary'], [20, 40, 60, 80])
m1, m2, m3, m4

(185480.0, 225000.0, 273598.0, 344600.0)

In [35]:
def percent(x):
  if x <= m1:
    return 1
  elif x > m1 and x <= m2:
    return 2
  elif x > m2 and x <= m3:
    return 3
  elif x > m3 and x <= m4:
    return 4
  else:
    return 5

In [36]:
df1['monetary_score'] = df1['monetary'].apply(percent)*coef.iloc[2,1]
df1.head(10)

,mem_no,last_ord_dt,recency,frequency,monetary,is_back,recency_score,frequency_score,monetary_score
0,1,01/01/2024,30,1,287000,0,1.217106,0.662923,0.266731
2,3,01/01/2024,30,1,548000,0,1.217106,0.662923,0.333413
3,4,27/01/2024,4,3,257000,1,4.868423,1.988768,0.200048
4,5,01/01/2024,30,2,65000,0,1.217106,1.325845,0.066683
5,6,15/01/2024,16,2,375000,1,3.651317,1.325845,0.333413
6,7,22/01/2024,9,4,925750,0,3.651317,2.651691,0.333413
8,9,01/01/2024,30,1,325000,0,1.217106,0.662923,0.266731
9,10,01/01/2024,30,1,229000,0,1.217106,0.662923,0.200048
10,11,23/01/2024,8,2,249000,1,4.868423,1.325845,0.200048
11,12,25/01/2024,6,2,185000,1,4.868423,1.325845,0.066683


In [37]:
df1['total_score'] = df1['recency_score'] + df1['frequency_score'] + df1['monetary_score']
df1.head(10)

,mem_no,last_ord_dt,recency,frequency,monetary,is_back,recency_score,frequency_score,monetary_score,total_score
0,1,01/01/2024,30,1,287000,0,1.217106,0.662923,0.266731,2.146759
2,3,01/01/2024,30,1,548000,0,1.217106,0.662923,0.333413,2.213442
3,4,27/01/2024,4,3,257000,1,4.868423,1.988768,0.200048,7.057239
4,5,01/01/2024,30,2,65000,0,1.217106,1.325845,0.066683,2.609634
5,6,15/01/2024,16,2,375000,1,3.651317,1.325845,0.333413,5.310576
6,7,22/01/2024,9,4,925750,0,3.651317,2.651691,0.333413,6.636421
8,9,01/01/2024,30,1,325000,0,1.217106,0.662923,0.266731,2.146759
9,10,01/01/2024,30,1,229000,0,1.217106,0.662923,0.200048,2.080076
10,11,23/01/2024,8,2,249000,1,4.868423,1.325845,0.200048,6.394316
11,12,25/01/2024,6,2,185000,1,4.868423,1.325845,0.066683,6.260951


In [38]:
df1.describe()

,mem_no,recency,frequency,monetary,is_back,recency_score,frequency_score,monetary_score,total_score
count,950.000000,950.000000,950.000000,950.000000,950.000000,950.000000,950.000000,950.000000,950.000000
mean,502.082105,21.774737,1.958947,275999.957895,0.260000,2.529018,1.284674,0.199627,4.013319
std,288.819598,10.735727,1.225562,143993.930960,0.438865,1.713174,0.770903,0.094501,2.325892
min,1.000000,0.000000,1.000000,65000.000000,0.000000,1.217106,0.662923,0.066683,1.946711
25%,251.250000,11.000000,1.000000,196670.000000,0.000000,1.217106,0.662923,0.133365,2.080076
50%,503.500000,30.000000,2.000000,250000.000000,0.000000,1.217106,1.325845,0.200048,2.709658
75%,752.750000,30.000000,3.000000,324247.500000,1.000000,3.651317,1.988768,0.266731,5.840133
max,1000.000000,30.000000,6.000000,2450000.000000,1.000000,6.085529,3.314614,0.333413,9.733556


In [39]:
#Classification into 5 groups by total score
t1, t2, t3, t4 = np.percentile(df1['total_score'], [20, 60, 75, 90])
t1, t2, t3, t4


(2.0133938092052786, 4.026787618410557, 5.840133404727579, 7.744787547315488)

In [40]:

def category(x) :
  if x <= t1 :
    return 5
  elif x > t1 and x <= t2 :
    return 4
  elif x > t2 and x <= t3 :
    return 3
  elif x > t3 and x <= t4 :
    return 2
  elif x > t4 :
    return 1

In [41]:
df1['category'] = df1['total_score'].apply(category)
df1

,mem_no,last_ord_dt,recency,frequency,monetary,is_back,recency_score,frequency_score,monetary_score,total_score,category
0,1,01/01/2024,30,1,287000,0,1.217106,0.662923,0.266731,2.146759,4
2,3,01/01/2024,30,1,548000,0,1.217106,0.662923,0.333413,2.213442,4
3,4,27/01/2024,4,3,257000,1,4.868423,1.988768,0.200048,7.057239,2
4,5,01/01/2024,30,2,65000,0,1.217106,1.325845,0.066683,2.609634,4
5,6,15/01/2024,16,2,375000,1,3.651317,1.325845,0.333413,5.310576,3
...,...,...,...,...,...,...,...,...,...,...,...
995,996,02/01/2024,29,3,310000,1,2.434212,1.988768,0.266731,4.689710,3
996,997,30/01/2024,1,2,164500,1,6.085529,1.325845,0.066683,7.478057,2
997,998,01/01/2024,30,1,136000,0,1.217106,0.662923,0.066683,1.946711,5
998,999,01/01/2024,30,1,142000,0,1.217106,0.662923,0.066683,1.946711,5


•	Customers were divided into five score categories based on their total RFM score using percentile-based thresholds : 1 - the best customers, 5-lost customers.

*   **Category 1** : Best Customers - Highest Engagement
*   **Category 2** : Loyal Customers
*   **Category 3** : Potential Loyal Customers
*   **Category 4** : At-Risk Customers
*   **Category 5** : Lost Customers

### **6. RFM Analysis Results Interpretation**


In [42]:
level_cnt = df1.groupby(['is_back', 'category'])[['mem_no']].count().reset_index()
level_cnt['is_back'] = level_cnt['is_back'].astype(str)

fig = px.bar(level_cnt, x='category', y='mem_no', color='is_back',
             width=800, height=500,
             template='plotly_white',
             color_discrete_sequence=['#EF553B','#636EFA'],  # Blue and Red
             labels={'mem_no': 'Number of customers', 'category': 'Score category', 'is_back': 'Returning status'},
             title='Number of Customers in Each Score Category',
             text=level_cnt['mem_no'].apply(lambda x: f'{x:.0f}'))

fig.update_traces(textposition='outside')
fig.show()

In [43]:
pivot = df1.groupby('category').agg({'is_back':'sum', 'mem_no':'count'}).reset_index()
pivot

,category,is_back,mem_no
0,1,68,89
1,2,84,142
2,3,64,138
3,4,31,390
4,5,0,191


In [44]:
pivot['retention_rate'] = pivot['is_back']/pivot['mem_no'] * 100
pivot

,category,is_back,mem_no,retention_rate
0,1,68,89,76.404494
1,2,84,142,59.154930
2,3,64,138,46.376812
3,4,31,390,7.948718
4,5,0,191,0.000000


In [45]:
pivot['churn_rate'] = 100 - pivot['retention_rate']
pivot

,category,is_back,mem_no,retention_rate,churn_rate
0,1,68,89,76.404494,23.595506
1,2,84,142,59.154930,40.845070
2,3,64,138,46.376812,53.623188
3,4,31,390,7.948718,92.051282
4,5,0,191,0.000000,100.000000


In [46]:
fig = px.bar(pivot, x='category', y='retention_rate', template='plotly_white',
             width=700, height = 500,
             color_discrete_sequence = px.colors.qualitative.Dark24,
             labels = dict(retention_rate = 'Retention Rate', category='Category'),
             title = 'Customer Retention Rate by Score Category',
             text=pivot['retention_rate'].apply(lambda x: f'{x:.0f}%'))

fig.update_traces(textposition='outside')
fig.show()

*   **Category 1** (Best Customers - Highest Engagement) : *Retention Rate* : 76% (Highest)
*   **Category 2** (Loyal Customers): *Retention Rate* : 59%
*   **Category 3** (Potential Loyal Customers): *Retention Rate* : 46%
*   **Category 4** (At-Risk Customers): *Retention Rate* : 8% (Extremely low)
*   **Category 5** (Lost Customers): *Retention Rate* : 0%


In [47]:
retained = df1[df1['is_back'] == 1]
retained.describe()

,mem_no,recency,frequency,monetary,is_back,recency_score,frequency_score,monetary_score,total_score,category
count,247.000000,247.000000,247.000000,247.000000,247.000000,247.000000,247.000000,247.000000,247.000000,247.000000
mean,466.396761,10.238866,3.085020,266429.352227,1.000000,4.351030,1.999504,0.200048,6.550582,2.234818
std,282.798825,8.465834,1.270706,98041.531417,0.000000,1.427424,0.748885,0.087544,1.792688,0.992616
min,4.000000,0.000000,2.000000,94500.000000,1.000000,2.434212,1.325845,0.066683,3.826740,1.000000
25%,217.500000,3.000000,2.000000,200000.000000,1.000000,2.434212,1.325845,0.133365,5.110528,1.000000
50%,452.000000,8.000000,3.000000,246670.000000,1.000000,4.868423,1.988768,0.200048,6.527682,2.000000
75%,707.500000,17.000000,4.000000,304500.000000,1.000000,6.085529,2.651691,0.266731,8.174321,3.000000
max,997.000000,29.000000,6.000000,777500.000000,1.000000,6.085529,3.314614,0.333413,9.733556,4.000000


In [56]:
fig = px.box(retained, x='category', y='monetary', color =retained['category'].astype(str),
             width=600, height = 500, template='plotly_white',
             labels = dict(category = 'Category', monetary='Spent amount'),
             title = 'Spending Distribution of Retained Customers',
                 )
fig.show()

In [63]:
fig = px.box(retained, x='category', y= 'frequency',
             color =retained['category'].astype(str),
             width=600, height = 500, template='plotly_white',
             labels = dict(category = 'Category', frequency='Frequency'),
             title = 'Frequency of Purchases Among Retained Customers',
                 )
# Ensure the y-axis displays whole numbers
fig.update_layout(yaxis=dict(tickmode='linear', dtick=1))

fig.show()

In [50]:
fig = px.box(retained, x='category', y='recency', color =retained['category'].astype(str),
             width=600, height = 500, template='plotly_white',
             labels = dict(category = 'Category', recency='Recency'),
             title = 'Purchase Recency of Retained Customers',
                 )
fig.show()

### **7. Analysis of churned customers**



In [51]:
fig = px.bar(pivot, x='category', y='churn_rate', template='plotly_white',
             width=700, height = 500,
             color_discrete_sequence = px.colors.qualitative.Light24,
             labels = dict(churn_rate = 'Churn Rate', category = 'Category'),
             title = 'Churn Rate by Customer Score Category',
             text=pivot['churn_rate'].apply(lambda x: f'{x:.0f}%'))

fig.update_traces(textposition='outside')
fig.show()

In [52]:
churned = df1[df1['is_back']==0]
churned.describe()

,mem_no,recency,frequency,monetary,is_back,recency_score,frequency_score,monetary_score,total_score,category
count,703.000000,703.000000,703.000000,703.000000,703.000000,703.000000,703.000000,703.000000,703.000000,703.000000
mean,514.620199,25.827881,1.563300,279362.603129,0.000000,1.888851,1.033518,0.199479,3.121848,3.911807
std,290.062272,8.210910,0.928293,156900.375629,0.000000,1.294286,0.603246,0.096886,1.768068,0.984591
min,1.000000,1.000000,1.000000,65000.000000,0.000000,1.217106,0.662923,0.066683,1.946711,1.000000
25%,260.500000,27.000000,1.000000,192000.000000,0.000000,1.217106,0.662923,0.133365,2.013394,4.000000
50%,525.000000,30.000000,1.000000,250000.000000,0.000000,1.217106,0.662923,0.200048,2.146759,4.000000
75%,769.500000,30.000000,2.000000,328500.000000,0.000000,2.434212,1.325845,0.266731,3.893422,5.000000
max,1000.000000,30.000000,6.000000,2450000.000000,0.000000,6.085529,3.314614,0.333413,9.666873,5.000000


In [53]:
churned.groupby('category')[['monetary']].agg(['sum', 'mean', 'median','std']).reset_index()

category   monetary                                          
                  sum          mean        median           std
0        1    6208120 295624.761905 282250.000000 135537.062814
1        2   16867090 290811.896552 254300.000000 134832.754187
2        3   19912100 269082.432432 234000.000000  96389.216611
3        4  120268000 335008.356546 299000.000000 180714.435313
4        5   33136600 173490.052356 179000.000000  35151.693161

In [54]:
churned.groupby('category')[['recency', 'frequency']].agg(['min', 'mean', 'max']).reset_index()

category recency               frequency             
               min      mean max       min     mean max
0        1       1  3.380952   8         3 3.666667   5
1        2       1  7.465517  24         2 2.879310   6
2        3       9 19.013514  30         2 2.770270   5
3        4      17 29.292479  30         1 1.278552   3
4        5      30 30.000000  30         1 1.000000   1

In [55]:
fig = px.box(churned, x='category', y='monetary', color = churned['category'].astype(str),
             width=600, height = 500, template='plotly_white',
             title = 'Churned Customers Spending Distribution',
             labels = dict(category = 'Category', monetary='Spent amount')
                 )
fig.show()